In [4]:
import pandas as pd

# ===============================
# 1. LOAD DATA
# ===============================
df_bebras = pd.read_csv("dashboard_bebras.csv", sep=';')
df_guru   = pd.read_csv("survei_literasidigital_ppks_guru.csv", sep=';')

In [5]:
# ===============================
# 2. NORMALISASI NAMA SEKOLAH
# ===============================
def norm(x):
    return str(x).strip().lower()

df_bebras["sekolah_norm"] = df_bebras["SekolahNama"].apply(norm)
df_guru["sekolah_norm"]   = df_guru["Asal Instansi"].apply(norm)

# ===============================
# 3. CARI SEKOLAH YANG SAMA
# ===============================
sekolah_siswa = set(df_bebras["sekolah_norm"].unique())
sekolah_guru  = set(df_guru["sekolah_norm"].unique())
sekolah_sama  = sekolah_siswa.intersection(sekolah_guru)

print("\nSEKOLAH YANG SAMA (ada di guru & siswa):")
for sekolah in sorted(sekolah_sama):
    print(f"   • {sekolah.title()}")


SEKOLAH YANG SAMA (ada di guru & siswa):
   • Sd Kristen Banjarsari
   • Sd Kristen Manahan Surakarta
   • Smk Kristen 1 Surakarta
   • Smp Kristen 1 Surakarta


In [6]:
# ===============================
# 4. AGREGASI DATA SISWA
# ===============================
siswa_agg = df_bebras.copy()

# pastikan nilai numerik
siswa_agg["Nilai"] = pd.to_numeric(siswa_agg["Nilai"], errors="coerce")

siswa_agg = siswa_agg.groupby("sekolah_norm").agg(
    jumlah_siswa=("Nama", "count"),
    rata_skor_siswa=("Nilai", "mean")
).reset_index()

siswa_agg["rata_skor_siswa"] = siswa_agg["rata_skor_siswa"].round(2)

# ===============================
# 5. AGREGASI DATA GURU
# ===============================
guru_agg = df_guru.groupby("sekolah_norm").size().reset_index(name="jumlah_guru")

# ===============================
# 6. MERGE DATA BERDASARKAN SEKOLAH
# ===============================
merged = pd.merge(guru_agg, siswa_agg, on="sekolah_norm", how="inner")
print("\nDATA MERGE SEKOLAH GURU & SISWA:")
print(merged)

# ===============================
# 7. HITUNG KORELASI
# ===============================
corr_jumlah = merged["jumlah_guru"].corr(merged["jumlah_siswa"])
corr_ct     = merged["jumlah_guru"].corr(merged["rata_skor_siswa"])

print("\nKorelasi jumlah guru vs jumlah siswa:", round(corr_jumlah, 3))
print("Korelasi jumlah guru vs rata skor CT siswa:", round(corr_ct, 3))


DATA MERGE SEKOLAH GURU & SISWA:
                   sekolah_norm  jumlah_guru  jumlah_siswa  rata_skor_siswa
0         sd kristen banjarsari           19            89             7.18
1  sd kristen manahan surakarta           35           219            12.80
2       smk kristen 1 surakarta           27            50             9.69
3       smp kristen 1 surakarta           22           215            16.63

Korelasi jumlah guru vs jumlah siswa: 0.375
Korelasi jumlah guru vs rata skor CT siswa: 0.23


In [7]:
# ===============================
# 8. RINGKASAN
# ===============================
print("\n===== RINGKASAN ANALISIS =====")
print(f"Total sekolah di data guru      : {df_guru['sekolah_norm'].nunique()}")
print(f"Total sekolah di data siswa     : {df_bebras['sekolah_norm'].nunique()}")
print(f"Sekolah yang sama               : {len(sekolah_sama)}")

print(f"\nKorelasi jumlah guru vs siswa   : {corr_jumlah:.3f}")
print(f"Korelasi jumlah guru vs CT      : {corr_ct:.3f}")


===== RINGKASAN ANALISIS =====
Total sekolah di data guru      : 15
Total sekolah di data siswa     : 21
Sekolah yang sama               : 4

Korelasi jumlah guru vs siswa   : 0.375
Korelasi jumlah guru vs CT      : 0.230
